In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
%%bash
pip install ipyleaflet pystac rich tqdm odc-geo pystac-client planetary-computer odc-stac catboost optuna pyarrow optuna-dashboard darts entropy scipy pyentrp

## Import Markdown

In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
# import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
import optuna


# Data Science
import numpy as np
import pandas as pd

# Feature Engineering
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD


# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix
from sklearn.manifold import TSNE

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('c861000c00fb430494b6ced2d9b15cf3')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
from catboost import CatBoostClassifier
tqdm.pandas()
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from catboost import CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer

# # Import everything from darts
# from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
# # from darts.dataprocessing.pipeline import Pipeline
# from darts import TimeSeries
# from darts.utils.timeseries_generation import gaussian_timeseries, linear_timeseries
# #from darts.models import RNNModel, TCNModel, TransformerModel, NBEATSModel, BlockRNNModel
# from darts.metrics import mape, smape

### Utilities function

In [ ]:
from scipy.stats import pearsonr
from pyentrp import entropy as ent
# Load data into a pandas DataFrame

# Calculate statistics for each column
def compute_stats(df) :
    stats = {}
    for col in df.columns:
        stats[f'min_{col}'] = df[col].min()
        stats[f'max_{col}'] = df[col].max()
        stats[f'range_{col}'] = df[col].max() - df[col].min()
        stats[f'mean_{col}'] = df[col].mean()
        if col != 'index':
            corr, _ = pearsonr(df[col], df['index'])
            stats[f'correlation_{col}'] = corr
            std_ts = df[col].std()
            stats[f'permutation_entropy_{col}'] = ent.permutation_entropy(df[col].values, order=4, delay=0.2*std_ts)

    return pd.DataFrame.from_dict(results, orient='index')

In [ ]:
def ordinal_distribution(data, dx=3, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None):
    '''
    Returns
    -------
     : tuple
       Tuple containing two arrays, one with the ordinal patterns occurring in data 
       and another with their corresponding probabilities.
       
    Attributes
    ---------
    data : array 
           Array object in the format :math:`[x_{1}, x_{2}, x_{3}, \\ldots ,x_{n}]`
           or  :math:`[[x_{11}, x_{12}, x_{13}, \\ldots, x_{1m}],
           \\ldots, [x_{n1}, x_{n2}, x_{n3}, \\ldots, x_{nm}]]`.
    dx : int
         Embedding dimension (horizontal axis) (default: 3).
    dy : int
         Embedding dimension (vertical axis); it must be 1 for time series 
         (default: 1).
    taux : int
           Embedding delay (horizontal axis) (default: 1).
    tauy : int
           Embedding delay (vertical axis) (default: 1).
    return_missing: boolean
                    If `True`, it returns ordinal patterns not appearing in the 
                    symbolic sequence obtained from **data** are shown. If `False`,
                    these missing patterns (permutations) are omitted 
                    (default: `False`).
    tie_precision : int
                    If not `None`, **data** is rounded with `tie_precision`
                    number of decimals (default: `None`).
   
    '''
    def setdiff(a, b):
        '''
        Returns
        -------
        : array
            An array containing the elements in `a` that are not contained in `b`.
            
        Parameters
        ----------    
        a : tuples, lists or arrays
            Array in the format :math:`[[x_{21}, x_{22}, x_{23}, \\ldots, x_{2m}], 
            \\ldots, [x_{n1}, x_{n2}, x_{n3}, ..., x_{nm}]]`.
        b : tuples, lists or arrays
            Array in the format :math:`[[x_{21}, x_{22}, x_{23}, \\ldots, x_{2m}], 
            \\ldots, [x_{n1}, x_{n2}, x_{n3}, ..., x_{nm}]]`.
        '''

        a = np.asarray(a).astype('int64')
        b = np.asarray(b).astype('int64')

        _, ncols = a.shape

        dtype={'names':['f{}'.format(i) for i in range(ncols)],
            'formats':ncols * [a.dtype]}

        C = np.setdiff1d(a.view(dtype), b.view(dtype))
        C = C.view(a.dtype).reshape(-1, ncols)

        return(C)

    try:
        ny, nx = np.shape(data)
        data   = np.array(data)
    except:
        nx     = np.shape(data)[0]
        ny     = 1
        data   = np.array([data])

    if tie_precision is not None:
        data = np.round(data, tie_precision)

    partitions = np.concatenate(
        [
            [np.concatenate(data[j:j+dy*tauy:tauy,i:i+dx*taux:taux]) for i in range(nx-(dx-1)*taux)] 
            for j in range(ny-(dy-1)*tauy)
        ]
    )

    symbols = np.apply_along_axis(np.argsort, 1, partitions)
    symbols, symbols_count = np.unique(symbols, return_counts=True, axis=0)

    probabilities = symbols_count/len(partitions)

    if return_missing==False:
        return symbols, probabilities
    
    else:
        all_symbols   = list(map(list,list(itertools.permutations(np.arange(dx*dy)))))
        miss_symbols  = setdiff(all_symbols, symbols)
        symbols       = np.concatenate((symbols, miss_symbols))
        probabilities = np.concatenate((probabilities, np.zeros(miss_symbols.__len__())))
        
        return symbols, probabilities

In [ ]:
def permutation_entropy(data, dx=3, dy=1, taux=1, tauy=1, base=2, normalized=True, probs=False, tie_precision=None):
    '''
    Returns Permutation Entropy
    Attributes:
    data : array
           Array object in the format :math:`[x_{1}, x_{2}, x_{3}, \\ldots ,x_{n}]`
           or  :math:`[[x_{11}, x_{12}, x_{13}, \\ldots, x_{1m}],
           \\ldots, [x_{n1}, x_{n2}, x_{n3}, \\ldots, x_{nm}]]`
           or an ordinal probability distribution (such as the ones returned by :func:`ordpy.ordinal_distribution`).
    dx :   int
           Embedding dimension (horizontal axis) (default: 3).
    dy :   int
           Embedding dimension (vertical axis); it must be 1 for time series (default: 1).
    taux : int
           Embedding delay (horizontal axis) (default: 1).
    tauy : int
           Embedding delay (vertical axis) (default: 1).
    base : str, int
           Logarithm base in Shannon's entropy. Either 'e' or 2 (default: 2).
    normalized: boolean
                If `True`, permutation entropy is normalized by its maximum value 
                (default: `True`). If `False`, it is not.
    probs : boolean
            If `True`, assumes **data** is an ordinal probability distribution. If 
            `False`, **data** is expected to be a one- or two-dimensional 
            array (default: `False`). 
    tie_precision : int
                    If not `None`, **data** is rounded with `tie_precision`
                    number of decimals (default: `None`).
    '''
    if not probs:
        _, probabilities = ordinal_distribution(data, dx, dy, taux, tauy, return_missing=False, tie_precision=tie_precision)
    else:
        probabilities = np.asarray(data)
        probabilities = probabilities[probabilities>0]

    if normalized==True and base in [2, '2']:        
        smax = np.log2(float(np.math.factorial(dx*dy)))
        s    = -np.sum(probabilities*np.log2(probabilities))
        return s/smax
         
    elif normalized==True and base=='e':        
        smax = np.log(float(np.math.factorial(dx*dy)))
        s    = -np.sum(probabilities*np.log(probabilities))
        return s/smax
    
    elif normalized==False and base in [2, '2']:
        return -np.sum(probabilities*np.log2(probabilities))
    else:
        return -np.sum(probabilities*np.log(probabilities))

In [ ]:
def generate_stastical_features(dataframe):
    '''
    Returns a  list of statistical features such as min,max,range,mean,auto-correlation,permutation entropy for each of the features
    Attributes:
    dataframe - DataFrame consisting of VV,VH and VV/VH for a time period
    '''
    features_list = []
    for index, row in dataframe.iterrows():
        for col in row: 
            min_vv = min(col)
            max_vv = max(col)
            range_vv = max_vv - min_vv
            mean_vv = np.mean(col)
            # print(correlation_vv)
            # permutation_entropy_vv = permutation_entropy(col, dx=6,base=2, normalized=True) 
            features_list.append([min_vv, max_vv, range_vv, mean_vv])
    
#         min_vh = min(row[1])
#         max_vh = max(row[1])
#         range_vh = max_vh - min_vh
#         mean_vh = np.mean(row[1])
#         correlation_vh = sm.tsa.acf(row[1])[1]
#         permutation_entropy_vh = permutation_entropy(row[1], dx=6, base=2, normalized=True)
    
#         min_vv_by_vh = min(row[2])
#         max_vv_by_vh = max(row[2])
#         range_vv_by_vh = max_vv_by_vh - min_vv_by_vh
#         mean_vv_by_vh = np.mean(row[2])
#         correlation_vv_by_vh = sm.tsa.acf(row[2])[1]
#         permutation_entropy_vv_by_vh = permutation_entropy(row[2], dx=6, base=2, normalized=True)
    return features_list

In [ ]:
def combine_two_datasets(dataset1,dataset2):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    data = pd.concat([dataset1,dataset2], axis=1)
    return data

In [ ]:
def get_sentinel_data(longitude, latitude, season,assests):
    
    '''
    Returns a list of VV,VH, VV/VH values for a given latitude and longitude over a given time period (based on the season)
    Attributes:
    longitude - Longitude
    latitude - Latitude
    season - The season for which band values need to be extracted.
    assets - A list of bands to be extracted
    
    '''
    
    bands_of_interest = assests
    if season == 'SA':
        time_slice = "2022-07-01/2022-07-31"
    if season == 'WS':
        time_slice = "2022-02-01/2022-02-28"
        
    vv_list = []
    vh_list = []
    red_list = []
    nir_list = []
    blue_list = []
    vv_by_vh_list = []
    
    bbox_of_interest = [longitude , latitude, longitude, latitude]
    time_of_interest = time_slice
    
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest)
    # search2 = catalog.search(collections=["alos-palsar-mosaic"], bbox=bbox_of_interest, datetime=time_of_interest)
    # search3 = catalog.search(collections=["modis-17A2HGF-061"], bbox=bbox_of_interest, datetime=time_of_interest)
    search4 = catalog.search(collections=["landsat-c2-l2"], bbox=bbox_of_interest, datetime=time_of_interest)
    items = list(search.get_all_items())
    items4 = list(search4.get_all_items())
    item = items[0]
    items.reverse()
    item4 = items4[0]
    items.reverse()
    
    data = stac_load([items[1]],bands=bands_of_interest, patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    selected_item = min(items4, key=lambda item: eo.ext(item).cloud_cover)
    # bands_of_interest = assests
    # data = stac_load([items[0]], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    # data2 = stac_load([items2[0]], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    # data3 = stac_load([items3[0]], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    data4 = stac_load([selected_item], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    
    for item in items:
        data = stac_load([item], bands=bands_of_interest, patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
        if(data['vh'].values[0][0]!=-32768.0 and data['vv'].values[0][0]!=-32768.0):
            data = data.where(~data.isnull(), 0)
            vh = data["vh"].astype("float64")
            vv = data["vv"].astype("float64")
            vv_list.append(np.median(vv))
            vh_list.append(np.median(vh))
            vv_by_vh_list.append(np.median(vv)/np.median(vh))
            
    for item in items4:
        data = stac_load([item], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
        if(data['red'].values[0][0]!=-32768.0 and data['blue'].values[0][0]!=-32768.0):
            data = data.where(~data.isnull(), 0)
            red = data["red"].astype("float64")
            blue = data["blue"].astype("float64")
            nir = data["nir08"].astype("float64")
            red_list.append(np.median(red))
            blue_list.append(np.median(blue))
            nir_list.append(np.median(nir))
              
    return vv_list, vh_list, vv_by_vh_list, red_list, blue_list, nir_list

In [ ]:
def get_sentinel_data(lat, long, season, assets):
    '''
    Returns VV and VH values for a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the VV and VH values have to be extracted
    assets - A list of bands to be extracted
    '''
    # latlong=latlong.replace('(','').replace(')','').replace(' ','').split(',')
    bbox_of_interest = [long , lat, long, lat]
    if season == 'SA':
        time_slice = "2022-07-01/2022-07-31"
    if season == 'WS':
        time_slice = "2022-02-01/2022-02-28"
    time_of_interest = time_slice
    time_of_interest2 = "2020-01-01/2020-12-31"
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    search = catalog.search(
        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    search2 = catalog.search(
        collections=["alos-palsar-mosaic"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    # Extract year and month from format 2023-01-01
    search3 = catalog.search(
        collections=["modis-17A2HGF-061"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    search4 = catalog.search(
        collections=["landsat-c2-l2"], bbox=bbox_of_interest, datetime=time_of_interest2,
        query={"eo:cloud_cover": {"lt": 10}},
    )
    items = list(search.get_all_items())
    items2 = list(search2.get_all_items())
    items3 = list(search3.get_all_items())
    items4 = list(search4.item_collection())
    selected_item = min(items4, key=lambda item: eo.ext(item).cloud_cover)
    print('items', items)
    print('items2', items2)
    print('items3', items3)
    print('items4', items4)
    # bands_of_interest = assests
    data = stac_load([items[0]], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    # data2 = stac_load([items2[0]], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    # data3 = stac_load([items3[0]], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    data4 = stac_load([selected_item], patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
    vh = data["vh"].astype("float").values.tolist()[0][0]
    vv = data["vv"].astype("float").values.tolist()[0][0]
    # hh = data2["HH"].astype("float").values.tolist()[0][0]
    # hv = data2["HV"].astype("float").values.tolist()[0][0]
    # gpp = data3["Gpp_500m"].astype("float").values.tolist()[0][0]
    # psn = data3["PsnNet_500m"].astype("float").values.tolist()[0][0]
    nir = data4["nir08"].astype("float").values.tolist()[0][0]
    red = data4["red"].astype("float").values.tolist()[0][0]
    blue = data4["blue"].astype("float").values.tolist()[0][0]
    return vh,vv, nir, red, blue




In [ ]:
def generate_feature_name(start_df):
    columns = []
    for i in start_df.columns: 
        columns = columns + [f'min_{i}', f'max_{i}', f'range_{i}', f'mean_{i}']
    return columns
        

### Retrieve the input csv

## We do not allow to use season, latlong, district name as out predictor

In [ ]:
crop_yield_data = pd.read_csv("/content/drive/MyDrive/Untitled Folder 1/Crop_Yield_Data_challenge_2.csv")
crop_yield_data.head()

In [ ]:
crop_yield_data.groupby('District').head()

In [ ]:
def retrieve_data(input_df) :
    assests = ['vh','vv']
    train_band_values=crop_yield_data.progress_apply(lambda x: get_sentinel_data(x['Latitude'],x['Longitude'], x['Season(SA = Summer Autumn, WS = Winter Spring)'],assests), axis=1)
    vh = [x[0] for x in train_band_values]
    vv = [x[1] for x in train_band_values]
    # vv_by_vh = [x[2] for x in train_band_values]
    red = [x[3] for x in train_band_values]
    blue = [x[4] for x in train_band_values]
    nir = [x[5] for x in train_band_values]
    vh_vv_data = pd.DataFrame(list(zip(vh,vv, red, blue, nir)),columns = ["vv_list","vh_list","red","blue","nir"])
    return vh_vv_data

In [ ]:
# vh_vv_data = retrieve_data(crop_yield_data)

### Do some statistical plotting

In [ ]:
# vh_vv_data.to_parquet('/content/drive/MyDrive/Untitled Folder 1/vh_vv.gzip', compression='gzip', index = False)

Start Here if finished downloading

In [ ]:
vh_vv_data = pd.read_parquet('/content/drive/MyDrive/Untitled Folder 1/vh_vv.gzip')

In [ ]:
vh_vv_data.info()

## Feature Engineering

In [ ]:
def reshape_list_columns(df):
    """
    Reshape any columns containing lists into multiple columns.
    Each element in the original lists will become a new column
    with a name based on the original column name and the element index.
    
    Parameters:
        df (pandas.DataFrame): the DataFrame to reshape
        
    Returns:
        pandas.DataFrame: the reshaped DataFrame
    """
    # Loop through each column in the DataFrame
    for col in df.columns:
        print(col)
        # Check if the column contains lists
        if df[col].dtype == 'object':
            print('Reach here')
            # Determine the maximum number of elements in the lists
            max_len = df[col].apply(len).max()
            # Create new column names for each element in the lists
            new_cols = [f"{col}_{i+1}" for i in range(max_len)]
            # Create a new DataFrame with the reshaped lists
            new_df = pd.DataFrame(df[col].to_list(), columns=new_cols)
            # Concatenate the new DataFrame with the original DataFrame
            df = pd.concat([df, new_df], axis=1)
            # Drop the original column from the DataFrame
            df.drop(col, axis=1, inplace=True)

    return df

In [ ]:
def ndvi(row):
    nir = row['nir_1']
    red = row['red_1']
    return (nir - red) / (nir + red)

# Apply the function to each row in the DataFrame

In [ ]:
def evi(row):
    nir = row['nir_1']
    red = row['red_1']
    blue = row['blue_1']
    return 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1))

# Apply the function to each row in the DataFrame


In [ ]:
def generate_trainning_dataset(input_df) :
    avoid_list = ['Season(SA = Summer Autumn, WS = Winter Spring)','Rice Crop Intensity(D=Double, T=Triple)','Latitude','Longitude', 'Date of Harvest']
    name_cols = list()
    for i in input_df.columns : 
        if i in avoid_list:
            continue
        name_cols.append(i)
    return input_df[name_cols]

In [ ]:
def feature_engineering(parquet, crop_yield_data):
    res = reshape_list_columns(parquet).dropna(axis=1)
    res['ndvi'] = res.apply(ndvi, axis=1)
    res['evi'] = res.apply(evi, axis=1)
    crop_yield_data['Date of Harvest'] = pd.to_datetime(crop_yield_data['Date of Harvest'], format='%d-%m-%Y')
    res = combine_two_datasets(res, crop_yield_data)
    dummy_df = pd.get_dummies(res['Rice Crop Intensity(D=Double, T=Triple)'], prefix='Rice Crop Intensity')
    res = combine_two_datasets(res, dummy_df)
    res = generate_trainning_dataset(res)
    return res
    

In [ ]:
trainning_df = feature_engineering(vh_vv_data, crop_yield_data)

In [ ]:
district_groups = trainning_df.groupby('District')

In [ ]:
district_dfs = [group_df for district, group_df in district_groups]

In [ ]:
len(district_dfs)

In [ ]:
## heatmeap to see the correlation between features. 
# Generate a mask for the upper triangle (taken from seaborn example gallery)
mask = np.zeros_like(district_dfs[0].corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.set_style('whitegrid')
plt.subplots(figsize = (15,12))
sns.heatmap(district_dfs[0].corr(), annot=True, mask = mask,
            cmap = 'RdBu', ## in order to reverse the bar replace "RdBu" with "RdBu_r"
            linewidths=.9, linecolor='white', fmt='.2g', center = 0, square=True)
plt.title("Correlations Among Features", y = 1.03,fontsize = 20, pad = 40)

## Handling outlier

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for i in district_dfs :
  sns.set(style="ticks")
  sns.pairplot(i)
  plt.show()  

In [ ]:
def replace_outliers_with_mean(df):
    avoid_dtypes = ['object','Timestamp']
    for col in df.columns:
        if col in ['red_1','nir_1','blue_1'] :
            df[col] = np.where(df[col] == 0, df[col].mean(), df[col])
        if df[col].dtype not in avoid_dtypes:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            upper_bound = q3 + 1.5 * iqr
            lower_bound = q1 - 1.5 * iqr
            df[col] = np.where(df[col] > upper_bound, df[col].mean(), df[col])
            df[col] = np.where(df[col] < lower_bound, df[col].mean(), df[col])
    return df

In [ ]:
def replace_outliers_zscore(df, threshold=3):
    """
    Replace outliers in a pandas DataFrame using the Z-score method with mean value of the column.
    
    Parameters:
    df (pandas DataFrame): The DataFrame to clean.
    threshold (float): The Z-score threshold for outlier detection. Data points with a Z-score greater than the
        threshold will be replaced. Default is 3.
    
    Returns:
    pandas DataFrame: The cleaned DataFrame with outliers replaced.
    """
    df_cleaned = df.copy()
    for col in df_cleaned.columns:
        zscore = np.abs((df_cleaned[col] - df_cleaned[col].mean()) / df_cleaned[col].std())
        mean_value = df_cleaned[col].mean()
        df_cleaned.loc[zscore > threshold, col] = mean_value
    return df_cleaned

In [ ]:
def normalize_df_arr(input_arr, outlier_replacement) :
  res = []
  for i in input_arr :
    tmp = i.drop('District', axis = 1)
    res.append(outlier_replacement(tmp))
  return res

In [ ]:
normalize_dfs = normalize_df_arr(district_dfs, replace_outliers_zscore)

In [ ]:
# normalize_df['ndvi'] = normalize_df.apply(ndvi, axis=1)
# normalize_df['evi'] = normalize_df.apply(evi, axis=1)

In [ ]:
# sns.set(style="ticks")
# sns.pairplot(normalize_df_2)
# plt.show()

In [ ]:
X = normalize_dfs[0].drop('Rice Yield (kg/ha)', axis=1)  # Features
y = normalize_dfs[0]['Rice Yield (kg/ha)']  # Target variable

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
X_train.shape[1]

In [ ]:
from datetime import datetime

In [ ]:

def objective(trial):
    
    # scalers = trial.suggest_categorical("scalers", ['standard','minmax','robust',None])
    # perform feature selection
    feature_select = trial.suggest_categorical("feature_selector", ["selectkbest", "none"])
    
    featureK= trial.suggest_int("k", 1, X_train.shape[1])
    print(f'featurek{featureK}')
    if feature_select == "selectkbest":
        feature_selector = SelectKBest(score_func=f_classif ,k=featureK)
    else :
        feature_selector = 'passthrough'

    # (b) Define your scalers\
    scaler = MinMaxScaler()
    # if scalers == "minmax":
    #     scaler = MinMaxScaler()
    # elif scalers == "standard":
    #     scaler = StandardScaler()
    # elif scalers == "robust" :
    #     scaler = RobustScaler()
    # else:
    #     scaler = 'passthrough'
    
    # -- Instantiate dimensionality reduction
     # (a) List all dimensionality reduction options
    dim_red = trial.suggest_categorical("dim_red", ["PCA","TruncatedSVD", None])
    n_components=trial.suggest_int("n_components", 1, featureK)
    # (b) Define the PCA algorithm and its hyperparameters
    if featureK == 1 or featureK >= n_components:
        dimen_red_algorithm='passthrough'
    else:
        if dim_red == "PCA":
            dimen_red_algorithm=PCA(n_components=n_components)
        elif dim_red == "TruncatedSVD":
            dimen_red_algorithm=TruncatedSVD(n_components=n_components)
        # (c) No dimensionality reduction option
        else:
            dimen_red_algorithm='passthrough'
        
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    
    pipeline = Pipeline([
        ("impute",imp),
        ("scaler", scaler),
        ("feature_selector",feature_selector),
        ("dim_red", dimen_red_algorithm),
        #Can change if gpu support is implementted
        ("catboost", CatBoostRegressor())
        
    ])
    
    # Parameter for tunning lightgbm
    params = {
        'catboost__iterations': 1000,
        'catboost__learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'catboost__depth': trial.suggest_int('depth', 4, 10),
        'catboost__l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        'catboost__bagging_temperature': trial.suggest_loguniform('bagging_temperature', 1e-3, 10),
        'catboost__border_count': trial.suggest_int('border_count', 32, 255),
        'catboost__verbose': False,
        'catboost__random_seed': 42
    }
    
    # params = {
    #     "catboost__iterations": trial.suggest_int("iterations", 100, 1000),
    #     "catboost__learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
    #     "catboost__depth": trial.suggest_int("depth", 4, 10),
    #     "catboost__l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-3, 10),
    #     "catboost__bagging_temperature": trial.suggest_loguniform("bagging_temperature", 0.1, 100),
    #     "catboost__random_strength": trial.suggest_loguniform("random_strength", 1e-3, 10),
    #     "catboost__border_count": trial.suggest_int("border_count", 32, 255),
    #     "catboost__thread_count": -1,
    #     "catboost__loss_function": "RMSE",
    #     "catboost__eval_metric": "RMSE",
    #     "catboost__verbose": False,
    # }

    # Fit the model
    model = pipeline.set_params(**params)
    # model.fit(X_train, y_train)
    # y_pred = model.predict(X_val)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    scores = r2_score(y_pred, y_test)
 
    try:
        scores = r2_score(y_pred, y_test)
        return scores
    except:
        return 0
    
study = optuna.create_study(direction='maximize',
                            storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
                            study_name=datetime.utcnow())

In [ ]:
study.optimize(objective, n_trials=500)

In [ ]:
   def print_res(input_study):
    print('Number of finished trials: {}'.format(len(input_study.trials)))
    print('Best trial:')
    trial = input_study.best_trial

    print('  Value: {}'.format(trial.value))
    print('  Params: ')

    for key, value in trial.params.items():
        print('    {}: {}'.format(key, value))

print_res(study)

In [ ]:
# Extract the result
def get_params(input_study) :
    params = {k: v for k, v in input_study.best_params.items() if k not in ('dim_red', 'scalers')}
    change = []
    for k,v in dict(params).items():
        tmp_name = k
        if 'catboost' not in tmp_name :
            res = f"catboost__{tmp_name}"
            params[res] = params.pop(tmp_name)
            change.append(res)
    params.pop('catboost__n_components')
    params.pop('catboost__k')
    params.pop('catboost__feature_selector')
    return params

params = get_params(study)

In [ ]:
params

In [ ]:
pipeline = Pipeline([
        ("impute",SimpleImputer(missing_values=np.nan, strategy='mean')),
        ("scaler", MinMaxScaler()),
        ("feature_selector",SelectKBest(score_func=f_classif ,k=5)),
        #Can change if gpu support is implementted
        ("catboost", CatBoostRegressor())
    ])

In [ ]:
pipeline.set_params(**params)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
# from sklearn.metrics import mean_absolute_percentage_error
# from sklearn.metrics import r2_score
# mean_absolute_percentage_error(y_pred, y_test)


In [ ]:
r2_score(y_pred, y_test)

### Generate the verification dataset

In [ ]:
result = pd.read_csv('/content/drive/MyDrive/Untitled Folder 1/Challenge_2_submission_template.csv')

In [ ]:
result.head()

In [ ]:
result.drop('Predicted Rice Yield (kg/ha)', axis = 1, inplace = True)

In [ ]:
# vh_vv_result_data = retrieve_data(result)

In [ ]:
# vh_vv_result_data.to_parquet('/content/drive/MyDrive/Untitled Folder 1/vh_vv_result_data.gzip', compression='gzip', index = False)

In [ ]:
vh_vv_result_data = pd.read_parquet('/content/drive/MyDrive/Untitled Folder 1/vh_vv_result_data.gzip')

In [ ]:
trainning_df = feature_engineering(vh_vv_result_data, result)

In [ ]:
final = trainning_df.drop(['ID No','District'], axis = 1)

In [ ]:
final.head()

In [ ]:
final_prediction_series = pd.Series(pipeline.predict(final))

In [ ]:
result['Predicted Rice Yield (kg/ha)'] = final_prediction_series

In [ ]:
result.head()

In [ ]:
result.to_csv('/content/drive/MyDrive/Untitled Folder 1/submission_not_tune.csv', index= False)